In [ ]:
import cv2
import numpy as np
from flask import Flask, request, render_template, jsonify
import face_recognition
import sqlite3

app = Flask(__name__)

known_faces = []
known_names = []


def load_known_faces_from_database():
    conn = sqlite3.connect('employee.db')
    cursor = conn.cursor()
    cursor.execute('SELECT name, face_encoding FROM employees')
    results = cursor.fetchall()
    for name, encoding_str in results:
        face_encoding = np.fromstring(encoding_str, dtype=np.float64, sep=',')
        known_faces.append(face_encoding)
        known_names.append(name)
    conn.close()

load_known_faces_from_database()

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        name = request.form['name']
        email = request.form['email']
        # Capture and save user images, generate unique ID, and add to the database
        user_id = add_employee_to_database(name, email)
        # Train the face recognition model with the new user's images
        train_face_recognition_model(user_id)
        return 'Registration successful!'
    return render_template('register.html')

def add_employee_to_database(name, email):
    conn = sqlite3.connect('employee.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO employees (name, email) VALUES (?, ?)', (name, email))
    conn.commit()
    user_id = cursor.lastrowid
    conn.close()
    return user_id

def train_face_recognition_model(user_id):
    pass

@app.route('/recognize', methods=['POST'])
def recognize():
    if request.method == 'POST':
        # Capture the image from the user's camera
        img = request.files['image'].read()
        nparr = np.fromstring(img, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # Detect faces in the image
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_img)
        face_encodings = face_recognition.face_encodings(rgb_img, face_locations)

        if not face_encodings:
            return jsonify({'status': 'error', 'message': 'No faces detected'})

        for face_encoding in face_encodings:
            # Compare the detected face with known faces
            matches = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.6)
            name = "Unknown"
            if True in matches:
                first_match_index = matches.index(True)
                name = known_names[first_match_index]

            if name != "Unknown":
                return jsonify({'status': 'success', 'message': 'User recognized', 'name': name})
            else:
                return jsonify({'status': 'error', 'message': 'Unknown user'})

if __name__ == '__main__':
    app.run(debug=True)